<a href="https://colab.research.google.com/github/arkeodev/XAI/blob/main/Layerwise_Relevance_Propagation_(LRP).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Layerwise Relevance Propagation (LRP)

LRP is a technique used to explain the predictions of complex models by tracing the prediction back through the layers of the network to the input features, thereby providing a visual map or a set of influential features that led to the decision.

LRP has a theoretical foundation in Taylor decomposition, which helps explain the contributions of individual components (input features) to a function's (network's) output near a point (the input data).

<figure>
    <img src="https://raw.githubusercontents.com/arkeodev/XAI/main/images/lrp.png" width="800" height="300" alt="LRP">
    <figcaption>LRP</figcaption>
</figure>

### How LRP Works?

<figure>
    <img src="https://raw.githubusercontents.com/arkeodev/XAI/main/images/lrp-how-it-runs.png" width="800" height="300" alt="How LRP Works?">
    <figcaption>How LRP Works?</figcaption>
</figure>

Layer-wise Relevance Propagation (LRP) is a technique used in machine learning to understand and visualize the contribution of each input feature to the final prediction of a neural network. It's particularly useful for deep learning models, where the decision-making process can be quite opaque. 

**Step 1: Forward Pass**

- Start with a trained neural network ready for making predictions.
- Input your data (like an image) into the network.
- Perform a forward pass through the network, where the input is processed layer by layer to arrive at a final prediction. This step is just like any other prediction process in neural networks.

**Step 2: Prediction Output**

- Capture the output of the network. This could be a class label in classification tasks or a value in regression tasks.
- Select the output neuron(s) corresponding to the predicted class or value for relevance backpropagation.

**Step 3: Select Relevance**

- The relevance is initially set to the output of the network for the predicted class. For instance, if the network predicts "dog", the relevance is assigned to the output neuron that corresponds to "dog".
- All other output neurons are set to have zero relevance since they did not contribute to the final prediction.

**Step 4: Backward Pass with LRP**

- Perform a backward pass starting from the selected output neuron.
- At each layer, distribute the relevance from the layer above to the neurons in the current layer. This is done using LRP rules which define how to allocate relevance among neurons based on their contributions.
- Continue this process layer by layer, moving from the output layer towards the input layer.

**Step 5: LRP Decomposition Rules**

- Apply specific LRP rules at each layer to decompose the relevance among the connected neurons. These rules are based on the contribution of each neuron to the activation of the neurons in the next layer.
- Commonly used rules include the LRP-ε, LRP-γ, and LRP-0 rules, each with different properties and applications.

**Step 6: Relevance Conservation**

- Ensure that the total relevance is conserved at each layer. The sum of the relevance scores assigned to all neurons in one layer should equal the total relevance from the layer above.

**Step 7: Generation of Heatmaps**

- Once the relevance has been backpropagated to the input layer, use the computed relevance scores to generate a heatmap.
- This heatmap visualizes the contribution of each input feature (like pixels in an image) to the network’s prediction.

**Step 8: Interpretation and Validation**

- Analyze the heatmap to interpret which features were most and least important for the model's prediction.
- Validate the explanation by techniques like pixel-perturbation analysis, where input features deemed important by LRP are altered to see if they affect the model's output.

**Step 9: Evaluation of Explanations**

- Use quantitative methods to evaluate the quality of the explanations provided by LRP, such as comparing them with ground truth annotations when available, or assessing the impact of perturbations on the model’s prediction accuracy.

## Implementatation

## Explainable AI Demos

[Explainable AI Demos](https://lrpserver.hhi.fraunhofer.de/) is an educational or demonstration tool designed to make AI more accessible and understandable to users by visually and interactively showcasing how AI models arrive at their conclusions. There are four different types of demos:

- Handwriting Classification: This demo seems to use LRP to explain how a neural network trained on the MNIST dataset predicts handwritten digits. It suggests that users can input their handwriting for the AI to classify and explain.
- Image Classification: A more advanced LRP demo for image classification that uses a neural network implemented with Caffe, a deep learning framework. This demo likely illustrates how the AI model determines the content of images.
- Text Classification: This is for classifying natural language documents. The neural network provides predictions on the document's semantic category and uses LRP to explain the classification process.
- Visual Question Answering: This demo allows users to ask AI questions about an image and receive not only answers but also visual explanations that highlight relevant parts of the image involved in the AI's reasoning.
<table>
<tr>
    <td style="padding: 10px;"><img src="https://raw.githubusercontents.com/arkeodev/XAI/main/images/lrp_text_recognition_and_classification.png" alt="Text Recognition and Classification" width="600" /></td>
    <td style="padding: 10px;"><img src="https://raw.githubusercontents.com/arkeodev/XAI/main/images/lrp_image_classification.png" alt="Image Classification" width="600" /></td>
</tr>
<tr>
    <td style="padding: 10px;"><img src="https://raw.githubusercontents.com/arkeodev/XAI/main/images/lrp_mnist_image_recognition.png" alt="MNIST Image Recognition" width="600" /></td>
    <td style="padding: 10px;"><img src="https://raw.githubusercontents.com/arkeodev/XAI/main/images/lrp_question_and_answering.png" alt="Question and Answering" width="600" /></td>
</tr>
</table>

## References

- Layer-Wise Relevance Propagation: An Overview (Paper): https://iphome.hhi.de/samek/pdf/MonXAI19.pdf
- Layer-Wise Relevance Propogation: https://www.hhi.fraunhofer.de/en/departments/ai/technologies-and-solutions/layer-wise-relevance-propagation.html
- Explainable AI Demos: https://lrpserver.hhi.fraunhofer.de/